

### About the DataSet

The full dataset includes:

 - 89,997,827 Customer documents
 - 89,997,827 IDcard documents
 - 179,998,862 Account documents
 - 179,998,862 Correspondance<State> documents
 - 179,998,862*60 Statement documents

Total = 89,997,827*2+ 179,998,862*62 = 11,339,925,098

The Data will be split in 3 repositories:

 - us-east
 	- 50% of Custoners/IDCards/Accounts...
 	- 6 months of statements
 - us-west
 	- 50% of Custoners/IDCards/Accounts...
 	- 6 months of statements
 - archives
 	- 54 months of statements for all custoners

This means:

 - us-east = 89,997,827 + 179,998,862 + 179,998,862/2*6 = 809,993,275
 - us-west = 89,997,827 + 179,998,862 + 179,998,862/2*6 = 809,993,275
 - archives = 179,998,862*54 = 9,719,938,548




### Step 0 : run DocumentProducers

#### Why making this a dedicated step?

There are several reasons to make DocumentProducer a dedicated step

 - generating 9+B documents randomly takes time (see below)
 - it can use a lot of CPU at the injector level and I would like to avoid having to run more than one injector
 - once we have all DocumentMessages in Kafka, we can snapshot this and run DocumentConsumers separately

#### Generation time

Based on the previous tests the Producers throughput is:

 - 80K docs/s for CSV based import
 - 40K docs/s for Random Generation 

So, the projected producer time is: 

 - Customers / Accounts hierarchy
 	- 2*(89,997,827 + 179,998,862)/80000 = 6749 s => 2h
 - Non Archived statements 
 	- (179,998,862*6)/40000 = 26999s => 8h
 - Archived statements
 	- (179,998,862*54)/40000 = 242998s => 68h

#### Generation steps

 - import/states-hierarchy
 - import/austomers
 - import/customers
 - import/statements_live
 - import/statements_archive0
 - import/statements_archive1
 - import/statements_archive2
 - import/statements_archive3
 - import/statements_archive4

### Step 1 : no Archives

#### Target volumes

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 0

#### Import steps

##### run DocumentProducers

The idea is to pre-produce all the DocumentMessages in Kafka before starting the import.

This 
 	- state
 - import initial structure
 - import customers
 - import accounts
 - import statements

Based on the previous tests the Producer throughput is:

 - 80K docs/s for CSV based import
 - 40K docs/s for Random Generation 

So, the projected producer time is: 

 - 2*(89,997,827 + 179,998,862)/80000 = 6749 s => 2h
 - (179,998,862*6)/40000 = 26999s => 8h



 - (179,998,862*54)/40000 = 26999s => 8h
